In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Output, Input
import  dash_bootstrap_components as dbc
pd.options.display.float_format = '{:,.0f}'.format

Carga de datos

In [2]:
df = pd.read_csv("D:/Zaeli/Analise Cliente Produto Comercial/Analisis Datos/VentasMesLineaProductoMunicipio.csv", parse_dates = ['Fecha'], dayfirst=True)

In [3]:
df = df.drop(columns=['Unnamed: 0', 'ChaveData'], axis=1)

In [4]:
opciones_estado = list(df.Estado.unique())
opciones_estado.sort()
opciones_estado.append('Todos')
opciones_linea = list(df.Linea.unique())
opciones_linea.sort()
opciones_linea.append('Todas')

Creacion del lay-out y los callbacks

In [5]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [6]:
app.layout = dbc.Container([
    dbc.Row([
        html.H3('Dashboard de Ventas', className='p-5 bg-light border')
    ]),
    dbc.Row([
        dbc.Col([
            html.H6('Estado'),
            dcc.Dropdown(id= 'seleccion_estado', options = opciones_estado, value='Todos', style={'width':'70%'}),
        ], width=5 ),
        dbc.Col([
            html.H6('Linea'),
            dcc.Dropdown(id= 'seleccion_linea', options = opciones_linea , value='Todas', style={'width':'70%'}),
        ], width=5),
    ]),
    dbc.Row([
        dcc.Graph(id='grafico_lineas' , figure={}),

    ]),
    dbc.Row([
        dcc.Graph(id='grafico_precio' , figure={}),
    ]),
    dbc.Row([
        dcc.Graph(id='grafico_margen' , figure={}),
    ]),

])

Grafico de evolucion del peso

In [7]:
@app.callback(
    Output('grafico_lineas', 'figure'),
    Input('seleccion_linea' , 'value'),
    Input('seleccion_estado', 'value')
)

def grafico_peso(linea, estado):
    if (linea == "Todas") & (estado == 'Todos'):
        dfT = df.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        fig = px.line(dfT,
                      x= "Fecha",
                      y = 'Peso',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Peso': ':,.0f',
                                  'Volume' : ':,.0f'},
                      title=f'Peso de {linea} en {estado} ')
        fig.update_traces(line=dict(color='royalblue', width=3))
    elif (linea != "Todas") & (estado == 'Todos'):
        df_filtrada = df.query('Linea ==@linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        fig = px.line(dfT,
                      x= "Fecha",
                      y = 'Peso',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Peso': ':,.0f',
                                  'Volume' : ':,.0f'},
                      title=f'Peso de {linea} en {estado} ')
        fig.update_traces(line=dict(color='royalblue', width=3))
    elif (linea == "Todas") & (estado != 'Todos'):
        df_filtrada = df.query('Estado ==@estado')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        fig = px.line(dfT,
                      x= "Fecha",
                      y = 'Peso',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Peso': ':,.0f',
                                  'Volume' : ':,.0f'},
                      title=f'Peso de {linea} en {estado} ')
        fig.update_traces(line=dict(color='royalblue', width=3))
    else:
        df_filtrada = df.query('Estado ==@estado & Linea==@linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        fig = px.line(dfT,
                      x= "Fecha",
                      y = 'Peso',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Peso': ':,.0f',
                                  'Volume' : ':,.0f'},
                      title=f'Peso de {linea} en {estado} ')
        fig.update_traces(line=dict(color='royalblue', width=3))
    return fig

Grafico de la evolucion del precio liquido

In [8]:
@app.callback(
    Output('grafico_precio', 'figure'),
    Input('seleccion_linea', 'value'),
    Input('seleccion_estado', 'value')
)

def grafico_precio(linea, estado):
    if (linea == 'Todas') & (estado == 'Todos'):
        dfT = df.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['PrecioKg'] = dfT['ReceitaBruta'] / dfT['Peso']
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'PrecioKg',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'PrecioKg':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Precio por kg de {linea} en {estado} ')
        fig.update_traces(line=dict(color='firebrick', width=3))
    elif (linea != 'Todas') & (estado == 'Todos'):
        df_filtrada = df.query('Linea==@linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['PrecioKg'] = dfT['ReceitaBruta'] / dfT['Peso']
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'PrecioKg',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'PrecioKg':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Precio por kg de {linea} en {estado} ')
        fig.update_traces(line=dict(color='firebrick', width=3))
    elif (linea == 'Todas') & (estado != 'Todos'):
        df_filtrada = df.query('Estado ==@estado')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['PrecioKg'] = dfT['ReceitaBruta'] / dfT['Peso']
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'PrecioKg',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'PrecioKg':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Precio por kg de {linea} en {estado} ')
        fig.update_traces(line=dict(color='firebrick',width=3))
    else:
        df_filtrada = df.query('Estado ==@estado & Linea == @linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['PrecioKg'] = dfT['ReceitaBruta'] / dfT['Peso']
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'PrecioKg',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'PrecioKg':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Precio por kg de {linea} en {estado} ')
        fig.update_traces(line=dict(color='firebrick', width=3))
    return fig


Grafico de la evolucion del margen unitario

In [9]:
@app.callback(
    Output('grafico_margen', 'figure'),
    Input('seleccion_linea', 'value'),
    Input('seleccion_estado' , 'value')
)

def grafico_precio(linea, estado):
    if (linea == 'Todas') & (estado == 'Todos'):
        dfT = df.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['Margen%'] = dfT['Margen'] / dfT['ReceitaBruta'] * 100
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'Margen%',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Margen%':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Margen de {linea} en {estado}' )
        fig.update_traces(line=dict(color='forestgreen', width=3))
    elif (linea != 'Todas') & (estado == 'Todos'):
        df_filtrada = df.query('Linea==@linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['Margen%'] = dfT['Margen'] / dfT['ReceitaBruta'] *100
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'Margen%',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Margen%':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Margen de {linea} en {estado}')
        fig.update_traces(line=dict(color='forestgreen', width=3))
    elif (linea == 'Todas') & (estado != 'Todos'):
        df_filtrada = df.query('Estado == @estado')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['Margen%'] = dfT['Margen'] / dfT['ReceitaBruta'] *100
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'Margen%',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Margen%':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Margen de {linea} en {estado}')
        fig.update_traces(line=dict(color='forestgreen', width=3))
    else:
        df_filtrada = df.query('Estado == @estado & Linea == @linea')
        dfT = df_filtrada.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha'], as_index=False).agg({
            'ReceitaBruta':'sum',
            'ReceitaLiquida':'sum',
            'Peso':'sum',
            'Volume':'sum',
            'Margen':'sum'})
        dfT['Margen%'] = dfT['Margen'] / dfT['ReceitaBruta'] *100
        fig = px.line(dfT,
                      x='Fecha',
                      y = 'Margen%',
                      hover_data={'ReceitaBruta':':,.0f',
                                  'Margen%':':,.2f',
                                  'Volume' : ':,.0f'},
                      title=f'Margen de {linea} en {estado}')
        fig.update_traces(line=dict(color='forestgreen', width=3))
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)